# Assignment 2
Collaborators (Name, study id, github handle): <br />
August Hertz Bugge - s194350 - libze<br />
David Bro Ludvigsen - s204102 - davidAlvaBro<br />
Sebastian Nicolai Fabricius Grut  - s204150 - Sebastiannfg

Github : https://github.com/davidAlvaBro/comsocsci2023.git 

#### Contributions 
We collaborated as a group. We have had weekly meetings where we completed the weekly assignments together - everything has been discussed and made together. 


In [10]:
# Imports that will be used through the notebook
import networkx as nx
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

DATA_PATH = "my_data/"

## 1 Mixing Patterns and Assortativity

### For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.

In [11]:
# Load the graph from week 4
CSS_graph = nx.read_graphml(DATA_PATH + 'CSS_graph.graphml')

In [3]:
# Function to calculate the fraction of neighbors that have the same attribute value
def get_matching_att(graph, att):
    """
    A function that calculates the fraction of neighbors that have the same attribute value
    
    Args:
        graph (nx.Graph): The graph to be analyzed
        att (str): The attribute to be analyzed
    
    Returns:
        match_frac (dict): A dictionary with the fraction of neighbors that have the same attribute value
    """
    match_frac = {}
    for node in graph.nodes:
        counter = 0
        neighbors = len(list(graph.neighbors(node)))

        if neighbors == 0: match_frac[node] = 0 # Have to check this, otherwise devising by zero

        else:
            for neighbor in graph.neighbors(node):
                # Check if the attribute value is the same for the node and the neighbor
                if nx.get_node_attributes(graph, att)[neighbor] == nx.get_node_attributes(graph, att)[node]:
                    counter += 1

            match_frac[node] = counter / neighbors

    return match_frac


In [4]:
# Get the average fraction of neighbors that have the same attribute value 
top_field_fracs = get_matching_att(CSS_graph, 'att2')

print(f"Average across all nodes {np.mean(list(top_field_fracs.values()))}")

Average across all nodes 0.3559081424843902


### Create a new graph, with the same nodes and edges, but where the association between nodes and field is shuffled. Compute the measure above for this randomized graph.

In [6]:
# Function that shuffles the attribute values of a graph
def shuffle_node_att(graph, att):
    """ 
    A function that shuffles the attribute values of a graph
    
    Args:
        graph (nx.Graph): The graph to be analyzed
        att (str): The attribute to be analyzed
    
    Returns: 
        graph (nx.Graph): The graph with shuffled attributes
    """
    # Make dictionary with shuffled attributes
    shuffled_atts = nx.get_node_attributes(graph, att)
    temp = list(shuffled_atts.values())
    random.shuffle(temp)
    new_atts = dict(zip(shuffled_atts, temp))

    # Set the shuffled attributes
    nx.set_node_attributes(graph, new_atts, att)
    
    return graph


In [8]:
# Make a copy of the graph and shuffle the attribute values
CSS_graph_copy = CSS_graph.copy()
CSS_graph_copy = shuffle_node_att(CSS_graph_copy, 'att2')

print(f"When the fields are randomly assigned, on average an auther has {np.mean(list(get_matching_att(CSS_graph_copy, 'att2').values()))} of their neighbors in the same field")


When the fields are randomly assigned, on average an auther has 0.2848694757379903 of their neighbors in the same field


### Repeat the point above 100 times (at least). Plot the distribution of the values obtained and compare it with the value you have found for the real graph. Is the chance to connect to a member of the same field significantly higher than it would be by chance?

In [12]:
# Shuffles the graph attributes N times and stores the distribution of the average fraction of neighbors that have the same attribute value
N = 300
distribution = np.zeros(N)
for i in tqdm(range(N)):
    CSS_graph_copy = shuffle_node_att(CSS_graph_copy, 'att2')
    avg = np.mean(list(get_matching_att(CSS_graph_copy, 'att2').values()))
    distribution[i] = avg 

np.save(DATA_PATH + "avg_same_field_distribution.npy", distribution)

  1%|▏         | 4/300 [27:30<33:54:14, 412.35s/it]

In [ ]:
# Load the distribution if not in memory 
if 'distribution' not in locals():
    distribution = np.load(DATA_PATH + "avg_same_field_distribution.npy")

In [ ]:
# Setup for plot
bins = 10
avg_value = np.mean(list(top_field_fracs.values()))

# Plot the distribution
figure, ax = plt.subplots(1)

ax.hist(distribution, density=True, bins=bins)
ax.axvline(x=avg_value, color='r', label=f"Mean value: {avg_value:.2f}")
ax.axvline(x=np.max(distribution), color='b', label=f"Minimum value: {np.max(distribution):.2f}")
ax.legend()
ax.title.set_text("Distribution of the average fraction of neighbors that have the same attribute value")

plt.show()

In the plot we see the distribution of the average fraction of coauthers of the same field each author has. The red line indicates our observed dataset as one datapoint, while the blue line indicates the largest value in the randomly generated datasets. 

We see that the observed average fraction of authors with the same field as the author in question is 0.356, but nothing close to this value has ever occured in the randomly generated data with 300 samples, this implies that the distribution of it stems from is significantly different than the randomly generated distribution. This is seen in the plot by the largest random value (the blue vertical line) being smaller than the observed value (the blue vertical line). 

In other words, authors tend to work together with authors of the same field. 